# Web Scraping de Pangoly
Este cuaderno tiene como objetivo recolectar datos a gran escala sobre placas base y sus componentes compatibles (Procesadores, Memoria RAM, Cajas y Disipadores) desde la web de Pangoly. Los datos extraidos se estructuran en un formato plano (JSONL) ideal para entrenar o afinar LLMs.

## 1. Importaciones y configuracion inicial
En esta primera celda, importamos las librerias necesarias. Destaca el uso de `undetected_chromedriver` para evitar bloqueos por sistemas anti-bots (como Cloudflare) y `BeautifulSoup` para analizar el HTML rapidamente. Tambien definimos la funcion que inicializa nuestro navegador fantasma y una funcion auxiliar para limpiar y convertir los precios a formato numerico.

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from bs4 import BeautifulSoup
import time
import random
import json

# Archivo de salida para los datos recolectados
OUTPUT_FILE = "pangoly.jsonl"

# Configura el undetected_chromedriver
def setup_driver():
    options = uc.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    driver = uc.Chrome(options=options)
    return driver

# Limpia el texto del precio y lo convierte en float
def parse_price(price_str):
    if not price_str or price_str == "N/A":
        return None
    
    # Eliminamos el simbolo de la divisa y posibles espacios
    cleaned = price_str.replace('€', '').strip()
    try:
        return float(cleaned)
    except ValueError:
        return None

## 2. Motores de extraccion de datos
Aqui definimos las dos funciones principales del "scraper":

1. **`get_motherboards_from_page`**: Navega por el catalogo general de placas base. Extrae el nombre, el enlace, el "slug" (identificador de la URL) y el precio base de cada placa.

2. **`get_compatible_components`**: Es el nucleo del scraper. Visita la pagina de cada categoria (CPU, RAM, etc) para una placa base especifica. Maneja la paginacion dinamica (AJAX) modificando el fragmento de la URL (`#page=X`) y utiliza "esperas de tiempo" (`WebDriverWait`) para asegurarse de que la tabla de compatibilidad se ha cargado en el navegador antes de extraer la marca, el modelo y el precio de los componentes.

In [ ]:
def get_motherboards_from_page(driver, page_num):
    url = f"https://pangoly.com/en/browse/motherboard?page={page_num}"
    driver.get(url)
    
    try:
        WebDriverWait(driver, 15).until(
            lambda d: d.find_elements(By.CSS_SELECTOR, "a.productItemLink")
        )
    except:
        return []

    time.sleep(random.uniform(2.0, 3.5))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    products = []
    
    items = soup.find_all('div', class_='productItem')
    for item in items:
        link_tag = item.find('a', class_='productItemLink')
        if not link_tag:
            continue
            
        link = link_tag.get('href')
        header = link_tag.find('header')
        name = header.text.strip() if header else link.split('/')[-1].replace('-', ' ').title()
        
        if link and not link.startswith('http'):
            link = f"https://pangoly.com{link}"
        slug = link.split('/')[-1]
        
        price_tag = item.find('span', title='Price')
        raw_price = price_tag.text.strip() if price_tag else "N/A"
        parsed_price = parse_price(raw_price)
        
        products.append({
            "name": name, 
            "url": link, 
            "slug": slug, 
            "price": parsed_price
        })
            
    return products

def get_compatible_components(driver, slug, category, max_category_pages=None):
    base_url = f"https://pangoly.com/en/compatibility/{slug}/{category}"
    compatible_items = {} 
    current_page = 1
    
    while True:
        url = base_url if current_page == 1 else f"{base_url}#page={current_page}"
        driver.get(url)
        
        if current_page > 1:
            time.sleep(1.0)
            driver.refresh()
            
        try:
            def is_page_ready(d):
                if d.find_elements(By.CSS_SELECTOR, "div.alert-info:not(.hidden)"): 
                    return True
                active_page = d.find_elements(By.CSS_SELECTOR, "ul.pagination li.active span")
                if active_page and active_page[0].text.strip() == str(current_page):
                    if d.find_elements(By.CSS_SELECTOR, "table.table-striped tbody tr"):
                        return True
                if current_page == 1:
                    pagination = d.find_elements(By.CSS_SELECTOR, "ul.pagination")
                    if not pagination and d.find_elements(By.CSS_SELECTOR, "table.table-striped tbody tr"):
                        return True
                return False

            WebDriverWait(driver, 15).until(is_page_ready)
            time.sleep(random.uniform(1.5, 3.0))
            
        except Exception:
            print(f"[!] Timeout o fin en {category} - Pagina {current_page}.")
            break
            
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        alert = soup.find('div', class_='alert-info')
        if alert and 'hidden' not in alert.get('class', []):
            break
            
        table = soup.find('table', class_='table-striped')
        if not table or not table.find('tbody'):
            break
            
        rows = table.find('tbody').find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            if len(cols) > 1:
                link_tag = cols[1].find('a')
                if link_tag and link_tag.find('strong'):
                    model = link_tag.find('strong').text.strip()
                    brand = ""
                    for child in cols[1].contents:
                        if child.name is None and child.strip():
                            brand = child.strip()
                            break 
                            
                    full_name = f"{brand} {model}" if brand else model
                    
                    raw_price = "N/A"
                    price_td = row.find('td', attrs={'data-label': 'Price'})
                    if price_td:
                        strong_price = price_td.find('strong')
                        if strong_price:
                            raw_price = strong_price.text.strip()
                            
                    parsed_price = parse_price(raw_price)
                            
                    if full_name not in compatible_items:
                        compatible_items[full_name] = parsed_price
        
        if max_category_pages and current_page >= max_category_pages:
            print(f"(Limite de {max_category_pages} paginas alcanzado.)", end="")
            break

        next_li = soup.find('li', class_='PagedList-skipToNext')
        if not next_li or 'disabled' in next_li.get('class', []):
            break 
            
        current_page += 1
        
    return [{"name": name, "price": price} for name, price in compatible_items.items()]

## 3. Bucle principal y logica de guardado (Flat JSONL)
Esta es la celda de ejecucion principal. El codigo coordina la recoleccion, iterando sobre las placas base y buscando sus componentes compatibles.

**Caracteristicas clave de esta seccion:**
* **Configuracion de limites:** Puedes definir `max_pages_to_scrape` y `max_category_pages` para limitar la recoleccion o dejarlos en `None` para raspar la web entera de forma autonoma.

* **Guardado "aplanado" (Flat JSONL):** guarda una linea independiente por cada par placa-componente de manera incremental. Esto previene la perdida de datos y facilita la ingesta de los mismos por parte del modelo LLM.

* **Cierre seguro:** Esta programado para atrapar excepciones de red. Si el usuario cierra la ventana de Chrome manualmente, el script lo detectara, guardara el progreso actual y finalizara el proceso limpiamente sin mostrar errores en pantalla.

In [ ]:
def main():
    driver = setup_driver()
    categories_to_scrape = ['cpu', 'ram', 'case', 'cpu-cooler']
    
    # ---------------- IMPORTANTE!!! VARIABLES DE CONTROL DE LIMITES ------------------
    start_page = 1             # Pagina inicial para comenzar el scraping
    max_pages_to_scrape = None # 'None' = Placas bases infinitas
    max_category_pages = None  # 'None' = Componentes infinitos
    # ---------------------------------------------------------------------------------
    
    page_num = start_page
    
    try:
        while True:
            if max_pages_to_scrape and page_num >= start_page + max_pages_to_scrape:
                print(f"\nLimite de {max_pages_to_scrape} paginas del catalogo alcanzado.")
                break
                
            print(f"\nExtrayendo catalogo de placas base: pagina {page_num}.")
            
            try:
                motherboards = get_motherboards_from_page(driver, page_num)
            except WebDriverException:
                print("\n[!] El navegador se ha cerrado, el proceso de scraping se ha detenido.")
                break

            if not motherboards:
                print(f"\nNo se encontraron mas placas base en la página {page_num}. Catalogo completado!")
                break
            
            for mb in motherboards:
                try: 
                    price_display = f"€{mb['price']}" if mb['price'] is not None else "Sin stock/precio"
                    print(f"\n- {mb['name']} -")
                    
                    for category in categories_to_scrape:
                        print(f" Buscando: {category.ljust(12)}", end="", flush=True)
                        components = get_compatible_components(driver, mb['slug'], category, max_category_pages)
                        print(f"| {len(components)} modelos.")
                        
                        # --- GUARDADO APLANADO EN EL ARCHIVO JSONL ---
                        with open(OUTPUT_FILE, 'a', encoding='utf-8') as f:
                            for comp in components:
                                flat_record = {
                                    "motherboard": mb['name'],
                                    "motherboard_price": mb['price'],
                                    "currency": "EUR",
                                    "component_type": category,
                                    "component_name": comp['name'],
                                    "component_price": comp['price'],
                                    "compatible": True
                                }
                                f.write(json.dumps(flat_record, ensure_ascii=False) + '\n')
                        
                except Exception as e:
                    error_msg = str(e).lower()
                    if "disconnected" in error_msg or "closed" in error_msg or "not reachable" in error_msg or "refused" in error_msg:
                        print("\n[!] El navegador se ha cerrado, el proceso de scraping se ha detenido.")
                        return 
                        
                    print(f"\n[X] Error procesando placa {mb['name']}: {e}. Se ha saltado a la siguiente.")
                    continue 
            
            page_num += 1
                    
    except KeyboardInterrupt:
        print("\n[!] Proceso detenido por teclado (Ctrl+C), el proceso de scraping se ha detenido.")
    finally:
        try:
            driver.quit() 
        except:
            pass
        print("Scraping finalizado. Un saludo socio!")

if __name__ == "__main__":
    main()